In [1]:
import numpy as np 
import cv2
import os
import tensorflow as tf

from keras.applications.vgg16 import VGG16
from sklearn.model_selection import KFold
from tensorflow.keras.utils import plot_model

In [2]:
# Establecer los valores de OSNR para cada espaciamiento de canal
osnr = {
    "single_ch": [14.3, 15.3, 16.3, 17.8, 19.3, 21.3, 23.3, 26.3, 28.3, 31.3, 36.3],
    "18GHz_spacing": [18, 19, 20, 23, 25, 27, 30, 32, 35, 40],
    # "17.6GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "17GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16.5GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15.5GHz_spacing": [20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15GHz_spacing": [23, 25, 27, 30, 32, 35, 40],
}

In [3]:
# Arreglos para almacenar las imágenes y etiquetas
images = []
labels = []

# Cargar las imágenes y etiquetas
for spacing in osnr.keys():
    directory = f"constellations/{spacing}/"
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            path = os.path.join(directory, filename)
            image = cv2.imread(path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.GaussianBlur(image, (3, 3), 0)
            image = image / 255.0
            images.append(image)
            if spacing == "single_ch":
                labels.append(0)
            elif spacing == "18GHz_spacing":
                labels.append(0)
            elif spacing == "17.6GHz_spacing":
                labels.append(0)
            elif spacing == "17GHz_spacing":
                labels.append(1)
            elif spacing == "16.5GHz_spacing":
                labels.append(1)
            elif spacing == "16GHz_spacing":
                labels.append(1)
            elif spacing == "15.5GHz_spacing":
                labels.append(1)
            elif spacing == "15GHz_spacing":
                labels.append(1)

images = np.array(images).astype("float32")
labels = np.array(labels).astype("uint8")
input_shape = (images[0].shape[0], images[0].shape[1], 3)
num_classes = len(np.unique(labels))

In [4]:
print("Input shape:", input_shape)
print("Number of classes:", num_classes)

Input shape: (64, 64, 3)
Number of classes: 2


## Usar una capa de clasificación personalizada para VGG16

In [5]:
def create_neuronal_network():
    vgg16 = VGG16(include_top=False, weights="imagenet", input_shape=input_shape)

    for layer in vgg16.layers:
        layer.trainable = False

    model = tf.keras.models.Sequential()
    model.add(vgg16)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


In [6]:
kflod = KFold(n_splits=10, shuffle=True)
precision = []
perdida = []

for train_index, test_index in kflod.split(images):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = create_neuronal_network()
    model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
    precision.append(accuracy)
    perdida.append(loss)




Epoch 1/100


18/18 [==============================] - 8s 364ms/step - loss: 0.5962 - accuracy: 0.7213 - val_loss: 0.5158 - val_accuracy: 0.7460
Epoch 2/100
18/18 [==============================] - 6s 340ms/step - loss: 0.5495 - accuracy: 0.7354 - val_loss: 0.5050 - val_accuracy: 0.7460
Epoch 3/100
18/18 [==============================] - 6s 334ms/step - loss: 0.5544 - accuracy: 0.7354 - val_loss: 0.5183 - val_accuracy: 0.7460
Epoch 4/100
18/18 [==============================] - 6s 336ms/step - loss: 0.5379 - accuracy: 0.7549 - val_loss: 0.5726 - val_accuracy: 0.8095
Epoch 5/100
18/18 [==============================] - 6s 339ms/step - loss: 0.5607 - accuracy: 0.7549 - val_loss: 0.5169 - val_accuracy: 0.7302
Epoch 6/100
18/18 [==============================] - 6s 333ms/step - loss: 0.5503 - accuracy: 0.7549 - val_loss: 0.5279 - val_accuracy: 0.7302
Epoch 7/100
18/18 [==============================] - 6s 349ms/step - loss: 0.5416 - accuracy: 0.7566 - val_loss: 0.5104 - val_accuracy: 0

In [7]:
best_accuracy = max(precision)
best_accuracy_index = precision.index(best_accuracy)
best_loss = perdida[best_accuracy_index]

In [8]:
a = [best_accuracy, best_loss]
np.save(f"vgg16_network_{num_classes}classes.npy", a)

In [9]:
a

[0.8571428656578064, 0.4421164393424988]